In [1]:
from ignite.contrib.metrics.regression import *
from ignite.contrib.metrics import *
from ignite.handlers import *
from ignite.metrics import *
from ignite.engine import *
from ignite.utils import *

from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.nn as nn

from active_learning.active_learning import ActiveLearner
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

from collections import OrderedDict
from scipy import signal
import numpy as np
import datetime
import pickle
import torch
import time
import os
import gc

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 50
EPOCH = 200
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
HIDDEN_UNITS = [180, 200, 250, 80, 100, 120]
K_FOLD = 1
VISIBLE_NUM = 50
MAT_PATH = "C:/Users/HILAB_Labtop_02/Desktop/insung/ecg-dbn/data/mit.mat"

PATH = "./pickle/mit"
DB1_LIST = [101, 106, 108, 109, 112, 114, 116, 118, 119, 122, 124, 201, 203, 205, 207, 208, 209, 215, 220, 221, 223, 230]
DB2_LIST = [100, 103, 105, 111, 113, 117, 121, 123, 200, 202, 210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234]

db1_sig, db1_ann = list(), list()
db1_resampled_sig, db1_resampled_ann = list(), list()
db3_resampled_sig, db3_resampled_ann = list(), list()

device = torch.device('cuda')
cpu = torch.device('cpu')
print(torch.cuda.get_device_name(device))

os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

2022-05-29 18:10:10.196055 model.py code start
NVIDIA GeForce RTX 3080 Laptop GPU


In [2]:
class RBM(nn.Module): 
    with torch.cuda.device(0):
        def __init__(self, n_vis, n_hid, k, batch):
            super(RBM, self).__init__()
            self.W      = nn.Parameter(torch.randn(1, batch, device=device) * 1e-2)
            self.n_vis  = n_vis
            self.n_hid  = n_hid
            self.k      = k
            self.batch  = batch
            self.v_bias = nn.Parameter(torch.zeros(n_vis, 1, device=device))
            self.h_bias = nn.Parameter(torch.zeros(n_hid, 1, device=device))
        
        def sample_from_p(self, p):
            return F.relu(
                torch.sign(
                    p - Variable(torch.randn(p.size(), device=device))
                )
            ).to(device=device)

        ''' ISSUE PART '''
        def v_to_h(self, v):
            w = (self.W.clone())
            # print("v -> h", v.size(), w.size())
            
            p_h = F.sigmoid(
                F.linear(v, w)
            ).to(device=device)

            sample_h = self.sample_from_p(p_h)
            return p_h, sample_h

        def h_to_v(self, h):
            w = self.W.t().clone()
            # print("h -> v", h.size(), w.size())

            p_v = F.sigmoid(
                F.linear(h, w)
            ).to(device=device)

            sample_v = self.sample_from_p(p_v)
            return p_v, sample_v
        
        def forward(self, v):
            pre_h1, h1 = self.v_to_h(v)
            h_ = h1

            for _ in range(self.k):
                pre_v_, v_ = self.h_to_v(h_)
                pre_h_, h_ = self.v_to_h(v_)
            return v, v_
        
        def get_weight(self):
            return self.W

In [3]:
def eval_step(engine, batch):
    return batch

default_model = nn.Sequential(OrderedDict([
    ('base', nn.Linear(4, 2)),
    ('fc', nn.Linear(2, 1))
]))

default_evaluator = Engine(eval_step)

def get_acc(y_true, y_pred):
    metric = Accuracy()
    metric.attach(default_evaluator, "accuracy")
    state = default_evaluator.run([[y_pred, y_true]])
    return state.metrics["accuracy"]
    
def to_categorical(y, num_classes):
    return np.eye(num_classes, dtype='uint8')[y]

def svm(x, w, b):
    h = (w*x).sum(1) + b
    return h


In [4]:
class SVM_Model(nn.Module):
    def __init__ (self, D_in, D_out):
        super(SVM_Model, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, D_out, device=device)
    
    def forward(self, x):
        out = self.linear1(x)
        return out

In [5]:
for i in range(len(DB1_LIST)):
    pickle_path = PATH + str(DB1_LIST[i]) + ".pkl"
    with open(pickle_path, 'rb') as f:
        db1_sig.append(pickle.load(f)[0])
        
    with open(pickle_path, 'rb') as f:
        db1_ann.append(pickle.load(f)[1])
        
print(len(db1_sig), len(db1_ann))

temp_sg, temp_ann = list(), list()

for i in range(len(db1_sig)):
    for j in range(len(db1_sig[i])):
        temp_sg.append(signal.resample(db1_sig[i][j], 50))
    db1_resampled_sig.append(temp_sg)

for i in range(len(db1_ann)):
    for j in range(len(db1_ann[i])):
        if db1_ann[i][j] == "N":
            # temp_ann.append(db1_ann[i][j])
            temp_ann.append(0)
        
        elif db1_ann[i][j] == "S" or "V" or "F":
            # temp_ann.append(db1_ann[i][j])
            temp_ann.append(1)
        
        else:
            temp_ann.append(1)
    
    db1_resampled_ann.append(temp_ann)

print(len(db1_resampled_sig), len(db1_resampled_ann))

22 22
22 22


In [6]:
PATH = "./svdb_pic/"

with open(PATH + 'RECORDS') as f:
    record_lines = f.readlines()

pre_records = []
for x in record_lines:
    pre_records.append(x.strip())
print("[RSLT]\t\t\t Export records ...")

for i in range(len(pre_records)):
    pickle_path = PATH + "svdb" + str(pre_records[i]) + ".pkl"
    with open(pickle_path, 'rb') as f:
        db1_sig.append(pickle.load(f)[0])
        
    with open(pickle_path, 'rb') as f:
        db1_ann.append(pickle.load(f)[1])
        
print(len(db1_sig), len(db1_ann))

temp_sg, temp_ann = list(), list()

for i in range(len(db1_sig)):
    for j in range(len(db1_sig[i])):
        temp_sg.append(signal.resample(db1_sig[i][j], 50))
    db3_resampled_sig.append(temp_sg)

for i in range(len(db1_ann)):
    for j in range(len(db1_ann[i])):
        if db1_ann[i][j] == "N":
            # temp_ann.append(db1_ann[i][j])
            temp_ann.append(0)
        
        elif db1_ann[i][j] == "S" or "V" or "F":
            # temp_ann.append(db1_ann[i][j])
            temp_ann.append(1)
        
        else:
            temp_ann.append("F")
    
    db3_resampled_ann.append(temp_ann)

print(len(db3_resampled_sig), len(db3_resampled_ann))

[RSLT]			 Export records ...
100 100
100 100


In [7]:
bbrbm_first = RBM(n_vis=VISIBLE_NUM, n_hid=HIDDEN_UNITS[3], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
bbrbm_second = RBM(n_vis=VISIBLE_NUM, n_hid=HIDDEN_UNITS[4], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
bbrbm_third = RBM(n_vis=VISIBLE_NUM, n_hid=HIDDEN_UNITS[5], k=K_FOLD, batch=BATCH_SIZE).to(device=device)

gbrbm_first = RBM(n_vis=VISIBLE_NUM, n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
gbrbm_second = RBM(n_vis=VISIBLE_NUM, n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
gbrbm_third = RBM(n_vis=VISIBLE_NUM, n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE).to(device=device)

first_train_op = optim.Adagrad(bbrbm_first.parameters(), LEARNING_RATE)
second_train_op = optim.Adagrad(bbrbm_second.parameters(), LEARNING_RATE)
third_train_op = optim.Adagrad(bbrbm_third.parameters(), LEARNING_RATE)

gb_first_train_op = optim.Adagrad(gbrbm_first.parameters(), LEARNING_RATE)
gb_second_train_op = optim.Adagrad(gbrbm_second.parameters(), LEARNING_RATE)
gb_third_train_op = optim.Adagrad(gbrbm_third.parameters(), LEARNING_RATE)

omse_loss = list()
output_gb = list()
best_acc = float()
svm_best_acc = float()
mse_loss = nn.MSELoss().to(device=device)
        
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

gaussian_std = torch.arange(1, 0, -0.02, device=device)
print(gaussian_std.size())

torch.Size([50])


In [8]:
train_dataloader = DataLoader(db1_resampled_sig,
                              batch_size=BATCH_SIZE,
                              num_workers=0,
                              shuffle=True)

print(len(train_dataloader.dataset))

test_dataloader = DataLoader(db3_resampled_sig,
                             batch_size=BATCH_SIZE,
                             num_workers=0,
                             shuffle=True)

22


In [10]:
'''BBRBM Train Part'''

loss_ = []
output_bb = []
temp_v1 = []
model_path_str = str()

print("RBM START!")

for epoch in range(EPOCH):
    tmp_acc = float()
    run_acc = float()
    start = time.time()

    for i, (data) in enumerate(train_dataloader):
        data = Variable(torch.tensor(data[0].to(device), dtype=torch.float32))
        sample_data = torch.normal(mean=data, std=gaussian_std).view(-1, 50).to(device=device)
        
        # tensor binary
        vog_first, v1 = gbrbm_first(sample_data)
        temp_v1.append(v1)
        
        omse_loss = mse_loss(vog_first, v1)
        
        gb_first_train_op.zero_grad()
        gb_first_train_op.step()
        omse_loss.backward()

    for _, (data) in enumerate(v1):         
        data = Variable(torch.tensor(data, dtype=torch.float32))
        sample_data = torch.normal(mean=data, std=gaussian_std).to(device=device)

        # tensor binary
        vog_second, v2 = gbrbm_second(sample_data)
        omse_loss = mse_loss(vog_second, v2)

        gb_second_train_op.zero_grad()
        omse_loss.backward()
        gb_second_train_op.step()

    for _, (data) in enumerate(v2):
        data = Variable(torch.tensor(data).type(torch.float32))
        sample_data = torch.normal(mean=data, std=gaussian_std).to(device=device)

        vog_third, v3_e = gbrbm_third(sample_data)
        omse_loss = mse_loss(vog_third, v3_e)
        
        gb_third_train_op.zero_grad()
        gb_third_train_op.step()
        omse_loss.backward()

        '''First bbrbm'''
        ''' ISSUE PART '''
        data = Variable(torch.tensor(v3_e, dtype=torch.float32)).uniform_(0, 1)
        sample_data = torch.bernoulli(data).to(device=device)
        
        # tensor binary
        fvog_first, v1 = bbrbm_first(sample_data)
        omse_loss = mse_loss(fvog_first, v1)

        first_train_op.zero_grad()
        first_train_op.step()
        omse_loss.backward()

        #  **** Second BBRBM **** #
        data = Variable(torch.tensor(v1, dtype=torch.float32)).uniform_(0, 1)
        sample_data = torch.bernoulli(data).to(device=device)

        # tensor binary
        vog_second, v2 = bbrbm_second(sample_data)
        omse_loss = mse_loss(vog_second, v2)

        second_train_op.zero_grad()
        omse_loss.backward()
        second_train_op.step()

        # **** THIRD BBRBM **** #
        data = Variable(torch.tensor(v2, dtype=torch.float32)).uniform_(0, 1)

        sample_data = torch.bernoulli(data).to(device=device)

        vog_third, v3 = bbrbm_third(sample_data)
        omse_loss = mse_loss(vog_third, v3)
        
        third_train_op.zero_grad()
        omse_loss.backward()
        third_train_op.step()

        run_acc += (sample_data == v3).sum().item()

        # for i in range(len(v3)):
        #     x = (v3.cpu().detach()).numpy()
        #     y = np.array(db1_resampled_ann)
            
        #     clf.fit(x, y)
        #     svm_pre = clf.predict(x, y)
            
    svm_pre = "?"
    acc_v = (vog_third >= 0).float()
    acc = get_acc(
        acc_v, v3
    ) * 100
    
    if acc > best_acc:
        best_acc = acc    
        
        path = "./say_cheese/ahh_saveMode_through_" + str(epoch) + "_" + str(acc) + "GBRBM.pth"
        model_path_str = path
        torch.save(gbrbm_third.state_dict(), path)
    output_gb.append(v3)

    print("GB-DBN Training loss for {0}th epoch {1}\tEstimate time : {2}\tAcc : {3}\t\tBest Acc : {4}\tSVM Predict : ".format(epoch + 1, omse_loss, time.time() - start, acc, best_acc, svm_pre))
    
    gc.collect()
print(v3.size())

RBM START!


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_16916/3339944148.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = Variable(torch.tensor(data[0].to(device), dtype=torch.float32))
c:\Users\HILAB_Labtop_02\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_16916/3339944148.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = Variable(torch.tensor(data, dtype=torch.float32))
C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_16916/3339944148.py:42: UserWarning: To copy 

GB-DBN Training loss for 1th epoch 0.4399999976158142	Estimate time : 3.7674779891967773	Acc : 70.0		Best Acc : 70.0	SVM Predict : 
GB-DBN Training loss for 2th epoch 0.7199999690055847	Estimate time : 2.9050354957580566	Acc : 72.0		Best Acc : 72.0	SVM Predict : 
GB-DBN Training loss for 3th epoch 0.5	Estimate time : 2.836604118347168	Acc : 64.0		Best Acc : 72.0	SVM Predict : 
GB-DBN Training loss for 4th epoch 0.47999998927116394	Estimate time : 2.792423725128174	Acc : 70.0		Best Acc : 72.0	SVM Predict : 
GB-DBN Training loss for 5th epoch 0.5999999642372131	Estimate time : 2.737701654434204	Acc : 74.0		Best Acc : 74.0	SVM Predict : 
GB-DBN Training loss for 6th epoch 0.5199999809265137	Estimate time : 3.058082103729248	Acc : 80.0		Best Acc : 80.0	SVM Predict : 
GB-DBN Training loss for 7th epoch 0.3999999761581421	Estimate time : 2.8283498287200928	Acc : 56.00000000000001		Best Acc : 80.0	SVM Predict : 
GB-DBN Training loss for 8th epoch 0.5199999809265137	Estimate time : 2.997875928

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

AL = ActiveLearner(strategy='entropy')
AL.rank(clf, db1_resampled_sig, num_queries=20)